In [1]:
import sys
import os
src_dir = os.path.join(os.getcwd(), '../src')
if src_dir not in sys.path:
    sys.path.append(src_dir)

from nn_model import *
from run_nn import *
from train_nn import *
from dataloader import *
from group_transformer import *

import pandas as pd

print(os.getcwd())


INFO:numexpr.utils:NumExpr defaulting to 8 threads.


/Users/isomorphicdude/Library/Mobile Documents/com~apple~CloudDocs/MSc/SB2.2_Statistical_Machine_Learning/Practicals/Assessed/SML_pratical/notebooks


In [2]:
# Load the training data and the test inputs
X_all = pd.read_csv('../data/X_train.csv', index_col = 0, header=[0, 1, 2]) # inputs of the training set
y_all = pd.read_csv('../data/y_train.csv', index_col = 0).squeeze('columns') # outputs of the training set
X_test = pd.read_csv('../data/X_test.csv', index_col = 0, header=[0, 1, 2]) # inputs of the test set

In [3]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
y_all = LabelEncoder().fit_transform(y_all)

In [4]:
from sklearn.model_selection import train_test_split
# create dataset
X_train, X_val, y_train, y_val = train_test_split(X_all, 
                                                  y_all, 
                                                  test_size=0.2, 
                                                  random_state=42,
                                                  shuffle=False)

# LDA fit on training data
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
LDA_model = LinearDiscriminantAnalysis(n_components=None)

X_train = X_train.to_numpy()
LDA_model.fit(X_train, y_train)

X_train = LDA_model.transform(X_train)

# LDA transform on validation data using training LDA model
X_val = X_val.to_numpy()

X_val = LDA_model.transform(X_val)

In [5]:
from torch.utils.data import DataLoader

train_set = MusicData(X_train, y_train)
val_set = MusicData(X_val, y_val)

train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
val_loader = DataLoader(val_set, batch_size=64, shuffle=True)

X shape: torch.Size([4800, 7])
y shape: torch.Size([4800])
X shape: torch.Size([1200, 7])
y shape: torch.Size([1200])


In [6]:
import numpy as np
mlp = MLP(input_dim=X_train.shape[1], 
          hidden_dims=[128, 64], 
          output_dim=np.unique(y_all).shape[0],
          dropout=0.0,
          activation = 'relu')

# train_loss, test_loss = run_training_loop(
#     num_epochs=10,
#     train_loader=train_loader,
#     val_loader=val_loader,
#     model=mlp,
#     optimizer=torch.optim.Adam(mlp.parameters(), lr=0.1),
#     loss_function=torch.nn.CrossEntropyLoss(),
#     device="cpu",
# )

In [ ]:
from sklearn.metrics import accuracy_score
# Load model state dict
mlp.load_state_dict(torch.load('models/model.pth'))
mlp.eval()

# predict
y_pred = mlp(torch.tensor(X_val).float()).detach().numpy()

accuracy_score(y_val, np.argmax(y_pred, axis=1))

0.5275